Actividad 2: Modelos basados en secuencias con Word2Vec
=======================================================

Introducción
------------

Los modelos basados en secuencias tienen la fortaleza que toman una secuencia de token (en un determinado orden) y generan una salida dependiendo del tipo de problema que se trate.
 - Seq2Class: Toman una secuencia de tokens y generan una clase
 - Seq2Seq: Toman una secuencia de token y generan otra secuencia de tokens. 
 
Vimos como podemos generar un modelo de secuencia utilizando `Word2Vec` y redes LSTM. Sin embargo ¿Les parece que conseguimos una buena performance?

En esta actividad les proponemos ver como podemos mejorar la performance de este modelo.

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv \
    --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/m72109/nlp/normalization.py \
    --quiet --no-clobber --directory-prefix ./m72109/nlp/
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/m72109/nlp/transformation.py \
    --quiet --no-clobber --directory-prefix ./m72109/nlp/
    
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/docs/nlp/neural/sequences-word2vec.txt \
    --quiet --no-clobber
!pip install -r sequences-word2vec.txt --quiet

Descargamos nuestros vectores de word2vec en español

In [ ]:
!mkdir -p ./Models/Word2Vec
!wget https://santiagxf.blob.core.windows.net/public/Word2Vec/model-es.bin \
    --quiet --no-clobber --directory-prefix ./Models/Word2Vec

In [1]:
import warnings
warnings.filterwarnings('ignore')

Instalamos las librerias necesarias

In [ ]:
!python -m spacy download es_core_news_sm

Cargamos el set de datos

In [1]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'], 
                                                    test_size=0.33, 
                                                    stratify=tweets['SECTOR'])

Direcciones
-----------

¿Como puede hacer para mejorar la performance del modelo original que creamos en clase? Explore diferentes alternativas que lo llevarán a una mejor performance. En particular:

- Remplazar la capa LSTM por una capa de tipo bidireccional. ¿Mejora?
- ¿Que sucede con el pre-procesamiento? ¿Serviría modificar algo?
    - Pista: Explore los parámteros de TweetNormalizer
    
Haga las modificaciones que crea pertinente y revise que propuestas mejoran la performance. Utilice la siguiente estructura de solución como ayuda, pero sientase libre de explorar otra.

> **Importante:** No es necesario realizar tuneo de hiper-parametros para resolver este ejercicio, solo utilice su intuición para introducir modificaciones que deberían de llevarlo a un mejor resultado.

Solución
---------

### Preprocesamiento de texto

In [3]:
from m72109.nlp.normalization import TweetTextNormalizer

In [4]:
normalizer = TweetTextNormalizer(preserve_case=False, return_tokens=True)

### Vectorización de las palabras

In [5]:
from m72109.nlp.transformation import Word2VecVectorizer

In [6]:
w2v = Word2VecVectorizer(model_path='Models/Word2Vec/model-es.bin', sequence_to_idx=True)
embedding_weights = w2v.get_weights()

### Construirmos un modelo basado en secuencias

#### Ajustando la longitud de las secuencias

In [7]:
from m72109.nlp.transformation import PadSequenceTransformer

In [8]:
max_seq_len = 100

In [9]:
seq2seq = PadSequenceTransformer(max_len=max_seq_len)

#### Construyendo el modelo

In [10]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, SpatialDropout1D

In [11]:
def build_model(sequence_len, vocab_size, emdedding_size, embedding_weights):
    model = Sequential([
        Embedding(vocab_size, emdedding_size,
                  weights=[embedding_weights],
                  trainable=False,
                  mask_zero=True),
        SpatialDropout1D(0.2),
        LSTM(emdedding_size),
        Dense(7, activation='softmax')
    ])
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Modelo:

In [12]:
estimator = keras.wrappers.scikit_learn.KerasClassifier(
    build_fn=build_model, 
    epochs=50,
    sequence_len=max_seq_len,
    vocab_size=w2v.vocab_size,
    emdedding_size=w2v.emdedding_size,
    embedding_weights=embedding_weights)

### Creando nuestro pipeline

In [13]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[('normalizer', normalizer), 
                           ('vectorizer', w2v),
                           ('padder', seq2seq),
                           ('estimator', estimator)])

In [ ]:
model = pipeline.fit(X=X_train, y=y_train)

### Evalución de los resultados

Probamos su performance utilizando el test set

In [ ]:
predictions = model.predict(X_test)

In [24]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

ALIMENTACION       0.96      0.85      0.90       110
  AUTOMOCION       0.87      0.78      0.82       148
       BANCA       0.84      0.82      0.83       198
     BEBIDAS       0.88      0.89      0.88       223
    DEPORTES       0.76      0.85      0.80       216
      RETAIL       0.79      0.80      0.80       268
       TELCO       0.95      0.94      0.94        79

    accuracy                           0.84      1242
   macro avg       0.86      0.85      0.85      1242
weighted avg       0.84      0.84      0.84      1242

